# Collection value evolution

Libs

In [1]:
import datetime as dt
import json
import numpy as np
import pandas as pd
import requests
import sqlite3
import yaml
from requests.exceptions import HTTPError
import time

Configuration file

In [2]:
with open(r'../config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

url_discogs_api = "https://api.discogs.com"

## Retrieve the collection items

Get number of pages of collection items

In [3]:
try:
    query = {'page': 1, 'per_page': 100}
    url_request = url_discogs_api + "/users/" + config["discogs_user"] + "/collection/folders/0/releases"
    response = requests.get(url_request, params=query)
    response.raise_for_status()
    jsonResponse = response.json()
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

no_pages = jsonResponse["pagination"]["pages"] # * 50

Get the collection items

In [10]:
collection_items = []
for i in range(1, no_pages + 1):
    try:
        query = {'page': i, 'per_page': 100}
        url_request = url_discogs_api + "/users/" + config["discogs_user"] + "/collection/folders/0/releases"
        response = requests.get(url_request, params=query)
        jsonResponse = response.json()
        collection_items.append(pd.json_normalize(jsonResponse["releases"]))
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

df_collection = pd.concat(collection_items, ignore_index=True)

In [11]:
df_collection.head()

,id,instance_id,date_added,rating,basic_information.id,basic_information.master_id,basic_information.master_url,basic_information.resource_url,basic_information.thumb,basic_information.cover_image,basic_information.title,basic_information.year,basic_information.formats,basic_information.labels,basic_information.artists,basic_information.genres,basic_information.styles
0,9438069,470722404,2020-05-16T20:50:48-07:00,0,9438069,417099,https://api.discogs.com/masters/417099,https://api.discogs.com/releases/9438069,,,Passenger,2016,"[{'name': 'Vinyl', 'qty': '1', 'descriptions':...","[{'name': '[PIAS]', 'catno': 'LHLP002', 'entit...","[{'name': 'Lisa Hannigan', 'anv': '', 'join': ...","[Pop, Folk, World, & Country]",[Folk]
1,9331164,877584547,2021-12-09T03:03:16-08:00,0,9331164,60848,https://api.discogs.com/masters/60848,https://api.discogs.com/releases/9331164,,,Sunset Mission,2016,"[{'name': 'Vinyl', 'qty': '2', 'descriptions':...","[{'name': '[PIAS]', 'catno': 'PIASD5013LP', 'e...","[{'name': 'Bohren & Der Club Of Gore', 'anv': ...","[Electronic, Jazz]",[Dark Jazz]
2,6239920,739755133,2021-06-24T16:19:33-07:00,0,6239920,281597,https://api.discogs.com/masters/281597,https://api.discogs.com/releases/6239920,,,Burma Shave,2008,"[{'name': 'DVD', 'qty': '1', 'text': '4:3', 'd...","[{'name': '3rd story production', 'catno': 'MC...","[{'name': 'Tom Waits', 'anv': '', 'join': '', ...","[Rock, Blues]",[]
3,1965832,470725568,2020-05-16T21:00:19-07:00,0,1965832,20096,https://api.discogs.com/masters/20096,https://api.discogs.com/releases/1965832,,,The Bad Seed,1983,"[{'name': 'Vinyl', 'qty': '1', 'descriptions':...","[{'name': '4AD', 'catno': '151.102', 'entity_t...","[{'name': 'The Birthday Party', 'anv': '', 'jo...","[Electronic, Rock]","[New Wave, Punk, Darkwave]"
4,403287,472111882,2020-05-20T19:39:29-07:00,5,403287,20838,https://api.discogs.com/masters/20838,https://api.discogs.com/releases/403287,,,Red Heaven,1992,"[{'name': 'Vinyl', 'qty': '1', 'descriptions':...","[{'name': '4AD', 'catno': 'CAD 2013', 'entity_...","[{'name': 'Throwing Muses', 'anv': '', 'join':...",[Rock],"[Indie Rock, Alternative Rock]"


Subselection of columns

In [12]:
selected_columns = df_collection.columns[~df_collection.columns.isin([ "basic_information.thumb", "basic_information.cover_image",\
     "basic_information.artists", "basic_information.labels", "basic_information.formats", "basic_information.genres",\ 
     "basic_information.styles"])]
df_collection = df_collection[selected_columns]

SyntaxError: unexpected character after line continuation character (1813903165.py, line 2)

Write stuff to sqlite db

In [7]:
db = sqlite3.connect("test.db")
df_collection.to_sql(name="collection", con=db, if_exists='replace')
db.close()

InterfaceError: Error binding parameter 11 - probably unsupported type.

## Retrieve marketplace values

In [ ]:
query = {'curr_abbr': 'EUR'}

collection_items_value = []
for i in df_collection.index:
    url_request = url_discogs_api + "/marketplace/stats/" + str(df_collection['id'][i])
    try:
        response = requests.get(url_request, params=query)
        response.raise_for_status()

        df_item = pd.json_normalize(response.json())
        df_item['id'] = str(df_collection['id'][i])
        df_item['time_value_retrieved'] = dt.datetime.now()
        df_item = df_item.loc[:, df_item.columns != 'lowest_price']
        collection_items_value.append(df_item)

    except HTTPError as http_err:
        if response.status_code == 429:
            time.sleep(60)
    except Exception as err:
        print(f'Other error occurred: {err}')
        
df_collection_value = pd.concat(collection_items_value, ignore_index=True)

In [ ]:
df_collection_value.head()

Add to database

In [ ]:
db = sqlite3.connect("test.db")
df_collection_value.to_sql(name="collection_value", con=db, if_exists='append')
db.close()